### Packages

In [1]:
using DynamicalSystems, DifferentialEquations, OrdinaryDiffEq, StaticArrays
using CairoMakie

### Functions

In [2]:
function model(u, p ,t)
    
    E, x, u_, y = u
    τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0 = p
    
    U(y, U0, ΔU0, ythr) = U0 + ΔU0 / ( 1 + exp( -50 * ( y - ythr ) ) )
    σ(x, xthr)= 1 / (1 + exp(-20 * (x - xthr)));
    
    du1 = (-E + α * log( 1 + exp( ( J * u_ * x * E + I0 ) / α ) )) / τ
    du2 = ( 1 - x ) / τ_D - u_ * x * E
    du3 = (U(y, U0, ΔU0, ythr) - u_) / τ_F + U(y, U0, ΔU0, ythr) * ( 1 - u_ ) * E
    du4 = -y /  τ_y + β * σ(x, xthr)
    
    return SVector(du1, du2, du3, du4)
    
end;

function output(index_i, index_j, U0_lc, I0_lc, u0_lc)
        println("index i: $index_i"); flush(stdout)
        println("index j: $index_j"); flush(stdout)
        println("U0: $U0_lc;"); flush(stdout)
        println("I0: $I0_lc;"); flush(stdout)
        println("u0: $u0_lc;"); flush(stdout)
end
function separate()
    println(" --------------------- "); flush(stdout)
    println("  "); flush(stdout)
end

function Y(x)
    a = 0.1836121365412579
    b = 0.2749462902187088
    c = 0.25
    return  a * (x.^2) + b * x .+ c 
end

function print_output(index, I0, U0, initial_condition)
    println("index: $index");flush(stdout)
    println("I0: $I0");flush(stdout)
    println("U0: $U0");flush(stdout)
    println("initial condition: $initial_condition")
end
function separate()
    println("_________________")
    println("")
end

separate (generic function with 1 method)

### Parametres

In [3]:
τ = 0.013; τ_D = 0.15; τ_y = 1.8; τ_F = 1;
α = 1.5; J = 3.07; β = 0.4375
ΔU0 = 0.305
xthr = 0.9; ythr = 0.5
parametres = SA[ τ, α, τ_D, J, NaN, ΔU0, τ_y, β, xthr, ythr, NaN ]
timefinal = 500.0
println(parametres)

[0.013, 1.5, 0.15, 3.07, NaN, 0.305, 1.8, 0.4375, 0.9, 0.5, NaN]


### Grid

In [4]:
startI = -1.8
endI = -1.24798
lengthI = 100

rangeI = range( startI, endI, length = lengthI )
rangeU0 = Y( rangeI );

In [5]:
file = "output_from_function.txt"

"output_from_function.txt"

In [ ]:
for (index, I0) in enumerate(rangeI)
    """
        Итерируемся по точкам c функции
    """
    if index == 1
        global initial_condition_function = SA[0.0, 0.2, 0.1, 0.1]
    end
    
    U0 = Y(I0)
    
    print_output(index, I0, U0, initial_condition_function)
    
    parametres = SA[  τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0  ]
    ds = ContinuousDynamicalSystem( model, initial_condition_function, parametres )
    
    tr = trajectory(ds, timefinal, Ttr = timefinal, Δt = 0.001;
                    diffeq = (alg = RK4(), adaptive = false, dense = false, dt = 0.001))
    
    initial_condition_function = tr[end, :]
    println("last point: $initial_condition_function");flush(stdout)
    λλ = lyapunovspectrum( ds, timefinal, Ttr = timefinal, Δt = 1;
                            diffeq = ( alg = RK4(), adaptive = false, dense = false, dt = 0.001 ))
    println("LSE: $λλ");flush(stdout)

    open(file, "a") do io
        println(io, " $I0 $U0 $(λλ[1])")
    end
    
    println("----------------------")
    if I0 == startI
        println("  ")
        println("FIRST ITERATION");flush(stdout)
        firstdot( I0, U0, initial_condition_function )
    end
    
    if I0 == endI
        println("  ")
        println("LAST ITERATION");flush(stdout)
        lastdot(I0, U0, initial_condition_function)
    end
    
    if startI < I0 < endI
        println("  ")
        println("$index ITERATION");flush(stdout)
        lastdot(I0, U0, initial_condition_function)
        firstdot( I0, U0, initial_condition_function )
    end
    
    separate()
end

index: 1
I0: -1.8
U0: 0.34999999999999987
initial condition: [0.0, 0.2, 0.1, 0.1]
last point: [1.9774203443433476, 0.8191410397203346, 0.7440336651009856, 0.039499383434498825]
LSE: [-0.00022855632034932043, -0.555555539508975, -2.2788049535087116, -8.775049309549539]
----------------------
  
FIRST ITERATION
  
 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 
MOVE TO RIGHT
->
index: 1
I0: -1.7944240404040406
U0: 0.34999999999999987
initial condition: [1.9774203443433476, 0.8191410397203346, 0.7440336651009856, 0.039499383434498825]
last point: [1.2803655145184345, 0.7987592975681482, 0.7613312732237268, 0.03512535069280929]
LSE: [0.001484303391490699, -0.5555555556801215, -2.321858223557142, -8.652236194996519]
_________________

->
index: 2
I0: -1.788848080808081
U0: 0.34999999999999987
initial condition: [1.2803655145184345, 0.7987592975681482, 0.7613312732237268, 0.03512535069280929]
last point: [0.9759435544199823, 0.7366430687949771, 0.7856605876013553, 0.032568412554895

last point: [28.29861655726123, 0.4821133760128753, 0.8078020781678754, 0.013389391134611182]
LSE: [0.007573097907152367, -0.5555555553781446, -2.926397318991242, -4.9787131150527975]
_________________

->
index: 23
I0: -1.6717529292929323
U0: 0.34999999999999987
initial condition: [28.29861655726123, 0.4821133760128753, 0.8078020781678754, 0.013389391134611182]
last point: [1.3552962832380655, 0.6073759063763854, 0.8219657263345472, 0.01129353314478279]
LSE: [0.0015223754392208171, -0.5555555555318349, -2.936281446343417, -4.743891778335746]
_________________

->
index: 24
I0: -1.6661769696969728
U0: 0.34999999999999987
initial condition: [1.3552962832380655, 0.6073759063763854, 0.8219657263345472, 0.01129353314478279]
last point: [1.897485211487522, 0.7519022330348588, 0.7818140923494131, 0.010935935202837456]
LSE: [-0.0038835620880160056, -0.5555555561464718, -2.939121246598831, -4.503414429785073]
_________________

->
index: 25
I0: -1.6606010101010134
U0: 0.34999999999999987
initi

In [7]:
function firstdot( I0, U0, initial_condition_function )
    
        println("  ");flush(stdout)
        println(" >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ");flush(stdout)
       
        println("MOVE TO RIGHT")
    
        step_to_right = rangeI.step.hi
        range_to_right_border = range( I0 + step_to_right, endI, step = step_to_right )
    
        for (index_right, I0_right) in enumerate(range_to_right_border)
        
            if index_right == 1
                global initial_condition_right = initial_condition_function
            end
            println("->")
            print_output(index_right, I0_right, U0, initial_condition_right)
        
            parametres = SA[  τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0_right  ]
        
            ds = ContinuousDynamicalSystem( model, initial_condition_right, parametres )
    
            tr = trajectory(ds, timefinal, Ttr = timefinal, Δt = 0.001;
                    diffeq = (alg = RK4(), adaptive = false, dense = false, dt = 0.001))
            initial_condition_right = tr[end, :]
        
            println("last point: $initial_condition_right");flush(stdout)
            λλ = lyapunovspectrum( ds, timefinal, Ttr = timefinal, Δt = 1;
                            diffeq = ( alg = RK4(), adaptive = false, dense = false, dt = 0.001 ))
            println("LSE: $λλ");flush(stdout)

            open(file, "a") do io
                println(io, " $I0_right $U0 $(λλ[1])")
            end
            separate()
        end
        println(" >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ");flush(stdout)
end

firstdot (generic function with 1 method)

In [6]:
function lastdot(I0, U0, initial_condition_function)
    
    println("  ");flush(stdout)
    println(" <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< ");flush(stdout)
    
    println("MOVE TO LEFT")
        
    step_to_left = rangeI.step.hi
    range_to_left_border = range( I0 - step_to_left, startI, step = -step_to_left )
        
    for (index_left, I0_left) in enumerate( range_to_left_border )
        
        if index_left == 1
            global initial_condition_left = initial_condition_function
        end
            
        println(" <- ")
        print_output(index_left, I0_left, U0, initial_condition_left)
        
        parametres = SA[  τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0_left  ]
        
        ds = ContinuousDynamicalSystem( model, initial_condition_left, parametres )
    
        tr = trajectory(ds, timefinal, Ttr = timefinal, Δt = 0.001;
                            diffeq = (alg = RK4(), adaptive = false, dense = false, dt = 0.001))
        initial_condition_left = tr[end, :]
        
        println("last point: $initial_condition_left");flush(stdout)
        λλ = lyapunovspectrum( ds, timefinal, Ttr = timefinal, Δt = 1;
                                diffeq = ( alg = RK4(), adaptive = false, dense = false, dt = 0.001 ))
        println("LSE: $λλ");flush(stdout)

        open(file, "a") do io
            println(io, " $I0_left $U0 $(λλ[1])")
        end
        separate()
    end
    println(" <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< ");flush(stdout)
end

lastdot (generic function with 1 method)